In [2]:
!pip install --upgrade pip
!pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1

    

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Using cached torch-1.13.1-cp37-cp37m-manylinux1_x86_64.whl (887.5 MB)
  Using cached torchdata-0.5.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.6 MB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
  Using cached portalocker-2.7.0-py2.py3-none-any.whl (15 kB)
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number

In [3]:
!pip install \
    transformers==4.27.2 \
    datasets==2.11.0 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0

  Using cached transformers-4.27.2-py3-none-any.whl (6.8 MB)
  Using cached datasets-2.11.0-py3-none-any.whl (468 kB)
  Using cached evaluate-0.4.0-py3-none-any.whl (81 kB)
  Using cached rouge_score-0.1.2-py3-none-any.whl
  Using cached loralib-0.1.1-py3-none-any.whl (8.8 kB)
  Using cached peft-0.3.0-py3-none-any.whl (56 kB)
  Obtaining dependency information for huggingface-hub<1.0,>=0.11.0 from https://files.pythonhosted.org/packages/7f/c4/adcbe9a696c135578cabcbdd7331332daad4d49b7c43688bc2d36b3a47d2/huggingface_hub-0.16.4-py3-none-any.whl.metadata
  Using cached huggingface_hub-0.16.4-py3-none-any.whl.metadata (12 kB)
  Using cached tokenizers-0.13.3-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
  Obtaining dependency information for tqdm>=4.27 from https://files.pythonhosted.org/packages/00/e5/f12a80907d0884e6dff9c16d0c0114d81b8cd07dc3ae54c5e962cc83037e/tqdm-4.66.1-py3-none-any.whl.metadata
  Using cached tqd

In [4]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

In [5]:
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)

Found cached dataset csv (/root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-391706c81424fc80/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
model_name = "google/flan-t5-base"
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype = torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f" Trainable model parameters : {trainable_model_params} \n All model parameters : {all_model_params} \n Percentage of trainable model parameters : {(trainable_model_params/all_model_params)*100} "
print(print_number_of_trainable_model_parameters(original_model))

 Trainable model parameters : 247577856 
 All model parameters : 247577856 
 Percentage of trainable model parameters : 100.0 


In [9]:
example_indices = [40,100]
dash_line = '-'.join("" for x in range(100))
for i,index in enumerate(example_indices):
    dialogue = dataset["test"][index]["dialogue"]
    summary = dataset["test"][index]["summary"]
    
    prompt = f"""
    summarize the following conversation.
    {dialogue}
    
    summary:"""
    
    inputs = tokenizer(prompt,return_tensors = 'pt')
    
    prediction = original_model.generate(inputs["input_ids"], max_new_tokens = 50)
    output = tokenizer.decode(prediction[0],skip_special_tokens = True)
    
    print(dash_line)
    print("EXAMPLE ", i+1)
    print(dash_line)
    print("Human Dialogue:")
    print(prompt)
    print(dash_line)
    print("Machine Summary - Zero Shot:")
    print(output)
    print(dash_line)
    print("Human Summary:")
    print(summary)

---------------------------------------------------------------------------------------------------
EXAMPLE  1
---------------------------------------------------------------------------------------------------
Human Dialogue:

    summarize the following conversation.
    #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
    
    summary:
---------------------------------------------------------------------------------------------------
Machine Summary - Zero Shot:
The train is about to leave.
---------------------------------------------------------------------------------------------------
Human Summary:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty

2. Perform Full Fine Tuning

2.1 Preprocess Dataset

In [10]:
def tokenize_function(example):
    start_prompt = "Summarize the following conversation. \n\n"
    end_prompt = "\n\n Summary:"
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    example["input_ids"] = tokenizer(prompt, padding = "max_length", truncation = True, return_tensors = "pt").input_ids
    example["labels"] = tokenizer(example["summary"], padding = "max_length", truncation = True, return_tensors = "pt").input_ids
    return example

tokenized_datasets= dataset.map(tokenize_function, batched = True)
tokenized_datasets = tokenized_datasets.remove_columns(["id","topic","dialogue","summary"])

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [11]:
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices = True)

Filter:   0%|          | 0/12460 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

In [12]:
print(f"Shapes of dataset\n")
print(f"Training set : {tokenized_datasets['train'].shape}\n")
print(f"Training set : {tokenized_datasets['validation'].shape}\n")
print(f"Training set : {tokenized_datasets['test'].shape}\n")

Shapes of dataset

Training set : (125, 2)

Training set : (5, 2)

Training set : (15, 2)



In [19]:
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 125
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 15
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 5
    })
})


In [22]:
output_dir = f'./dialogue_summary_trainig_{str(int(time.time()))}'
training_args = TrainingArguments(
            output_dir = output_dir,
            learning_rate = 1e-5,
            num_train_epochs = 1,
            weight_decay = 0.01,
            logging_steps = 1,
            max_steps = 1)
trainer = Trainer(
    model = original_model,
    args = training_args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["validation"])

In [27]:
trainer.train()
full_model_path = f"full_dialogue_summary_checkpoint_local"
trainer.model.save_pretrained(full_model_path)
tokenizer.save_pretrained(full_model_path)

Step,Training Loss
1,49.000000


('full_dialogue_summary_checkpoint_local/tokenizer_config.json',
 'full_dialogue_summary_checkpoint_local/special_tokens_map.json',
 'full_dialogue_summary_checkpoint_local/tokenizer.json')

In [26]:
!aws s3 cp --recursive s3://dlai-generative-ai/models/flan-dialogue-summary-checkpoint/ ./flan-dialogue-summary-checkpoint/
#Did this from cmd . Used the same command without first sign as it is windows

fatal error: An error occurred (IllegalLocationConstraintException) when calling the ListObjectsV2 operation: The unspecified location constraint is incompatible for the region specific endpoint this request was sent to.


In [5]:
!ls -alh ./flan_dialogue_summary_checkpoint/pytorch_model.bin

ls: cannot access './flan_dialogue_summary_checkpoint/pytorch_model.bin': No such file or directory


In [13]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained("./flan-dialogue-summary-checkpoint", torch_dtype = torch.bfloat16)

In [15]:
index = 200
dash_line = '-'.join("" for x in range(100))

dialogue = dataset["test"][index]["dialogue"]
human_summary = dataset["test"][index]["summary"]

prompt = f"""
summarize the following conversation.
{dialogue}

summary:"""

input_ids = tokenizer(prompt,return_tensors = 'pt').input_ids

original_model_outputs = original_model.generate(input_ids = input_ids, generation_config = GenerationConfig(max_new_tokens = 200, num_beams = 1))
original_model_text_outputs = tokenizer.decode(original_model_outputs[0],skip_special_tokens = True)

instruct_model_outputs = instruct_model.generate(input_ids = input_ids, generation_config = GenerationConfig(max_new_tokens = 200, num_beams = 1))
instruct_model_text_outputs = tokenizer.decode(instruct_model_outputs[0],skip_special_tokens = True)


print(dash_line)
print("EXAMPLE ", i+1)
print(dash_line)
print("Human Summary:")
print(human_summary)
print(dash_line)
print("Original model summary:")
print(original_model_text_outputs)
print(dash_line)
print("Instruct Model Summary:")
print(instruct_model_text_outputs)

---------------------------------------------------------------------------------------------------
EXAMPLE  2
---------------------------------------------------------------------------------------------------
Human Summary:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
Original model summary:
#Person1#: Have you considered upgrading your system?
---------------------------------------------------------------------------------------------------
Instruct Model Summary:
#Person1# suggests #Person2# adding a painting program to #Person2#'s software and upgrading the hardware. #Person2# also wants to add a CD-ROM drive.


Evaluate the model with Rouge metrics

In [16]:
rouge = evaluate.load('rouge')

#Test for 10 outputs

In [18]:

dash_line = '-'.join("" for x in range(100))

dialogues = dataset["test"][0:10]["dialogue"]
human_baseline_summaries = dataset["test"][0:10]["summary"]

original_model_summaries = []
instruct_model_summaries = []

for _,dialogue in enumerate(dialogues):
    prompt = f"""
    summarize the following conversation.
    {dialogue}

    summary:"""

    input_ids = tokenizer(prompt,return_tensors = 'pt').input_ids

    original_model_outputs = original_model.generate(input_ids = input_ids, generation_config = GenerationConfig(max_new_tokens = 200))
    original_model_text_outputs = tokenizer.decode(original_model_outputs[0],skip_special_tokens = True)
    original_model_summaries.append(original_model_text_outputs)

    instruct_model_outputs = instruct_model.generate(input_ids = input_ids, generation_config = GenerationConfig(max_new_tokens = 200))
    instruct_model_text_outputs = tokenizer.decode(instruct_model_outputs[0],skip_special_tokens = True)
    instruct_model_summaries.append(instruct_model_text_outputs)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries,instruct_model_summaries))
df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries'])
df

,human_baseline_summaries,original_model_summaries,instruct_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,The memo is to be distributed to all employees...,#Person1# asks Ms. Dawson to take a dictation ...
1,In order to prevent employees from wasting tim...,The memo is to be distributed to all employees...,#Person1# asks Ms. Dawson to take a dictation ...
2,Ms. Dawson takes a dictation for #Person1# abo...,The memo is to be distributed to all employees...,#Person1# asks Ms. Dawson to take a dictation ...
3,#Person2# arrives late because of traffic jam....,The traffic jam at the Carrefour intersection ...,#Person2# got stuck in traffic again. #Person1...
4,#Person2# decides to follow #Person1#'s sugges...,The traffic jam at the Carrefour intersection ...,#Person2# got stuck in traffic again. #Person1...
5,#Person2# complains to #Person1# about the tra...,The traffic jam at the Carrefour intersection ...,#Person2# got stuck in traffic again. #Person1...
6,#Person1# tells Kate that Masha and Hero get d...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced. Kate can'...
7,#Person1# tells Kate that Masha and Hero are g...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced. Kate can'...
8,#Person1# and Kate talk about the divorce betw...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced. Kate can'...
9,#Person1# and Brian are at the birthday party ...,"#Person1#: Happy Birthday, Brian. #Person2#: T...",Brian's birthday is coming. Brian dances with ...


In [19]:
original_model_results = rouge.compute(
    predictions = original_model_summaries,
    references = human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator = True,
    use_stemmer = True
)

instruct_model_results = rouge.compute(
    predictions = instruct_model_summaries,
    references = human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator = True,
    use_stemmer = True
)

print("Original Model Scores:")
print(original_model_results)
print(dash_line)
print("Instruct Model Scores:")
print(instruct_model_results)

Original Model Scores:
{'rouge1': 0.23972447825079402, 'rouge2': 0.10226305127006391, 'rougeL': 0.2046409608909609, 'rougeLsum': 0.20754735274472114}
---------------------------------------------------------------------------------------------------
Instruct Model Scores:
{'rouge1': 0.38881239922736843, 'rouge2': 0.17008025424479878, 'rougeL': 0.27974300959076787, 'rougeLsum': 0.27746115944731187}


In [20]:
improvement = (np.array(list(instruct_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(instruct_model_results.keys(), improvement):
    print(f'{key}:{value*100:.2f}%')

rouge1:14.91%
rouge2:6.78%
rougeL:7.51%
rougeLsum:6.99%


##PEFT Fine-Tuning

In [22]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r = 32,
    lora_alpha= 32,
    target_modules= ['q','v'],
    lora_dropout = 0.05,
    bias = 'none',
    task_type = TaskType.SEQ_2_SEQ_LM
)



In [23]:
peft_model = get_peft_model(original_model, lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

 Trainable model parameters : 3538944 
 All model parameters : 251116800 
 Percentage of trainable model parameters : 1.4092820552029972 


In [25]:
output_dir = f'./peft-dialogue-summary-trainig_{str(int(time.time()))}'
peft_training_args = TrainingArguments(
            output_dir = output_dir,
            auto_find_batch_size = True,
            learning_rate = 1e-3,
            num_train_epochs = 1,
            logging_steps = 1,
            max_steps = 1)

peft_trainer = Trainer(
    model = peft_model,
    args = peft_training_args,
    train_dataset = tokenized_datasets["train"],)

In [26]:
peft_trainer.train()

peft_model_path = f"peft-dialogue-summary-checkpoint-local"
peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:395: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Step,Training Loss
1,50.750000


('peft-dialogue-summary-checkpoint-local/tokenizer_config.json',
 'peft-dialogue-summary-checkpoint-local/special_tokens_map.json',
 'peft-dialogue-summary-checkpoint-local/tokenizer.json')

In [27]:
!aws s3 cp --recursive s3://dlai-generative-ai/models/peft-dialogue-summary-checkpoint/ ./peft-dialogue-summary-checkpoint-from-s3/

fatal error: An error occurred (IllegalLocationConstraintException) when calling the ListObjectsV2 operation: The unspecified location constraint is incompatible for the region specific endpoint this request was sent to.


In [28]:
!ls -alh ./peft-dialogue-summary-checkpoint-from-s3/adapter_model.bin

-rw-r--r-- 1 root root 14M Aug 19 20:30 ./peft-dialogue-summary-checkpoint-from-s3/adapter_model.bin


In [31]:
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype = torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
peft_model = PeftModel.from_pretrained(peft_model_base,
                                       './peft-dialogue-summary-checkpoint-from-s3',
                                       torch_dtype = torch.bfloat16,
                                       is_trainable = False)

In [32]:
print(print_number_of_trainable_model_parameters(peft_model))

 Trainable model parameters : 0 
 All model parameters : 251116800 
 Percentage of trainable model parameters : 0.0 


In [33]:
index = 200
dash_line = '-'.join("" for x in range(100))

dialogue = dataset["test"][index]["dialogue"]
human_summary = dataset["test"][index]["summary"]

prompt = f"""
summarize the following conversation.
{dialogue}

summary:"""

input_ids = tokenizer(prompt,return_tensors = 'pt').input_ids

original_model_outputs = original_model.generate(input_ids = input_ids, generation_config = GenerationConfig(max_new_tokens = 200, num_beams = 1))
original_model_text_outputs = tokenizer.decode(original_model_outputs[0],skip_special_tokens = True)

instruct_model_outputs = instruct_model.generate(input_ids = input_ids, generation_config = GenerationConfig(max_new_tokens = 200, num_beams = 1))
instruct_model_text_outputs = tokenizer.decode(instruct_model_outputs[0],skip_special_tokens = True)

peft_model_outputs = peft_model.generate(input_ids = input_ids, generation_config = GenerationConfig(max_new_tokens = 200, num_beams = 1))
peft_model_text_outputs = tokenizer.decode(peft_model_outputs[0],skip_special_tokens = True)

print(dash_line)
print("EXAMPLE ", i+1)
print(dash_line)
print("Human Summary:")
print(human_summary)
print(dash_line)
print("Original model summary:")
print(original_model_text_outputs)
print(dash_line)
print("Instruct Model Summary:")
print(instruct_model_text_outputs)
print(dash_line)
print("Peft Model Summary:")
print(peft_model_text_outputs)

---------------------------------------------------------------------------------------------------
EXAMPLE  2
---------------------------------------------------------------------------------------------------
Human Summary:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
Original model summary:
#Pork1: Have you considered upgrading your system? #Person1: Yes, I'm not sure. #Pork2: You're right. #Pork3: I'm not sure. #Pork3: I'm not sure. #Pork4: I'm not sure. #Pork5: I'm not sure. #Pork4: I'm not sure. #Pork3: I'm not sure. #Pork4: I'm not sure. #Pork4: I'm not sure. #Pork3: I'm not sure. #Pork4: I'm not sure. #Pork4: I'm not sure. #Pork4: I'm not sure. #Pork3: I'm not sure. #Pork4: I
---------------------------------------------------------------------------------------------------
Instruct Model Summary:
#Person1# suggests #Person2# adding a pa

In [34]:
dash_line = '-'.join("" for x in range(100))

dialogues = dataset["test"][0:10]["dialogue"]
human_baseline_summaries = dataset["test"][0:10]["summary"]

original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

for _,dialogue in enumerate(dialogues):
    prompt = f"""
    summarize the following conversation.
    {dialogue}

    summary:"""

    input_ids = tokenizer(prompt,return_tensors = 'pt').input_ids

    original_model_outputs = original_model.generate(input_ids = input_ids, generation_config = GenerationConfig(max_new_tokens = 200))
    original_model_text_outputs = tokenizer.decode(original_model_outputs[0],skip_special_tokens = True)
    original_model_summaries.append(original_model_text_outputs)

    instruct_model_outputs = instruct_model.generate(input_ids = input_ids, generation_config = GenerationConfig(max_new_tokens = 200))
    instruct_model_text_outputs = tokenizer.decode(instruct_model_outputs[0],skip_special_tokens = True)
    instruct_model_summaries.append(instruct_model_text_outputs)
    
    peft_model_outputs = peft_model.generate(input_ids = input_ids, generation_config = GenerationConfig(max_new_tokens = 200))
    peft_model_text_outputs = tokenizer.decode(peft_model_outputs[0],skip_special_tokens = True)
    peft_model_summaries.append(peft_model_text_outputs)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries,instruct_model_summaries, peft_model_summaries))
df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries', 'peft_model_summaries'])
df

,human_baseline_summaries,original_model_summaries,instruct_model_summaries,peft_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,The memo is a dictation for the office.,#Person1# asks Ms. Dawson to take a dictation ...,#Person1# asks Ms. Dawson to take a dictation ...
1,In order to prevent employees from wasting tim...,Employees are being required to take a dictati...,#Person1# asks Ms. Dawson to take a dictation ...,#Person1# asks Ms. Dawson to take a dictation ...
2,Ms. Dawson takes a dictation for #Person1# abo...,This memo is a memo to all employees in the of...,#Person1# asks Ms. Dawson to take a dictation ...,#Person1# asks Ms. Dawson to take a dictation ...
3,#Person2# arrives late because of traffic jam....,Person1: You're finally here!,#Person2# got stuck in traffic again. #Person1...,#Person2# got stuck in traffic and got stuck i...
4,#Person2# decides to follow #Person1#'s sugges...,The person has been waiting for a long time.,#Person2# got stuck in traffic again. #Person1...,#Person2# got stuck in traffic and got stuck i...
5,#Person2# complains to #Person1# about the tra...,The car is causing a traffic jam at work.,#Person2# got stuck in traffic again. #Person1...,#Person2# got stuck in traffic and got stuck i...
6,#Person1# tells Kate that Masha and Hero get d...,Masha and Hero are getting a divorce.,Masha and Hero are getting divorced. Kate can'...,Kate tells #Person2# Masha and Hero are gettin...
7,#Person1# tells Kate that Masha and Hero are g...,Masha and Hero are having a divorce.,Masha and Hero are getting divorced. Kate can'...,Kate tells #Person2# Masha and Hero are gettin...
8,#Person1# and Kate talk about the divorce betw...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced. Kate can'...,Kate tells #Person2# Masha and Hero are gettin...
9,#Person1# and Brian are at the birthday party ...,"#Porce1: Brian, thank you for coming to our pa...",Brian's birthday is coming. Brian dances with ...,Brian remembers his birthday and invites #Pers...


In [35]:
original_model_results = rouge.compute(
    predictions = original_model_summaries,
    references = human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator = True,
    use_stemmer = True
)

instruct_model_results = rouge.compute(
    predictions = instruct_model_summaries,
    references = human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator = True,
    use_stemmer = True
)

peft_model_results = rouge.compute(
    predictions = peft_model_summaries,
    references = human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator = True,
    use_stemmer = True
)

print("Original Model Scores:")
print(original_model_results)
print(dash_line)
print("Instruct Model Scores:")
print(instruct_model_results)
print(dash_line)
print("Peft Model Scores:")
print(peft_model_results)

Original Model Scores:
{'rouge1': 0.23405506287085232, 'rouge2': 0.07366666666666669, 'rougeL': 0.21329166934430094, 'rougeLsum': 0.21648051753314912}
---------------------------------------------------------------------------------------------------
Instruct Model Scores:
{'rouge1': 0.38881239922736843, 'rouge2': 0.17008025424479878, 'rougeL': 0.27974300959076787, 'rougeLsum': 0.27746115944731187}
---------------------------------------------------------------------------------------------------
Peft Model Scores:
{'rouge1': 0.3636453677458462, 'rouge2': 0.12664656829900028, 'rougeL': 0.2703710325071955, 'rougeLsum': 0.2701132504457793}


In [37]:
!touch ./.gitignore